### <span style="color:red">本案例创建环境须创建10G以上的EVS配置</span>

| 参数 | 说明 |
| - - - - - | - - - - - |
| 计费方式 | 按需计费  |
| 名称 | Notebook实例名称，如 nlp_bert_squad |
| 工作环境 | Python3 |
| 资源池 | 选择"公共资源池"即可 |
| 类型 | GPU |
| 规格 | GPU:1*p100 CPU:8核64GiB |
| 存储配置 | 选择EVS，磁盘规格10GB |

## BERT模型在SQuAD数据集上的示例 

在官方的BERT代码中，就提供了一个**run_squad.py**示例程序，在此我们尝试在SQuAD数据集上运行BERT问答模型任务。

### 关于SQuAD数据集

[SQuAD](https://rajpurkar.github.io/SQuAD-explorer/)（Stanford Question Answering Dataset）语料库为斯坦福大学的Rajpurkar等人于2016年在自然语言处理的顶级会议[EMNLP](https://www.aclweb.org/mirror/emnlp2016/)（Empirical Methods in Natural Language Processing）上发布，是行业内公认的机器阅读理解领域的顶级水平测试，给定一篇文章，准备相应问题，需要算法给出问题的答案。此数据集所有文章选自维基百科，数据集的量为当今其他数据集(例如，WikiQA)的几十倍之多。

#### 数据集的具体构建如下

1.	文章是随机选取的wiki百科，一共有536篇wiki被选中。而每篇wiki，会被切成段落，最终生成了23215个自然段。之后就对这23215个自然段进行阅读理解，对提出的107,785个问题进行自动问答。

2.	之后斯坦福利用众包的方式，进行了给定文章，提问题并给答案的人工标注。将两万多个段落给不同标注人员，要求对每个段落提五个问题。

3.	让另一些标注人员对提的这个问题用文中最短的片段给予答案，如果不会或者答案没有在文章中出现可以不给。之后经过验证，标注人员所提的问题在问题类型分布上足够多样，并且有很多需要推理的问题，也就意味着这个集合十分有难度。如下图所示，作者列出了该数据集答案的类别分布，我们可以看到19.8%的答案为日期和其他数字类；有32.6%的答案为三种不同类型的专有名词：人名、地名以及其它实体对象；31.8%为名词短语；其余15.8%由形容词短语、动词短语、从句和其它类型组成，各类别且比例相当。

![数据集2.png](./img/数据集2.png)
 
4.	这个数据集的评测标准有两个，第一：F1（模糊匹配），第二：EM（Exact Match，完全匹配）。EM表示必须机器给出的和人给出的一样才算正确。哪怕有一个字母不一样，也会算错。而F1是将答案的短语切成词，和人的答案一起算recall，Precision和F1，即如果匹配了一些词但不全对，仍然算分。
下图为SQuAD语料库中的一个原文-问题-答案样本。
 
 ![数据集1.png](./img/数据集1.png)
 
### 准备数据集

在本实践中，主要使用较小的SQuAD v1.1数据集，所需训练集和验证集可以在此下载：

- [train-v1.1.json](https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json)
- [dev-v1.1.json](https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json)

该训练集和验证集JSON文件解析如下
 
 ![数据集3.png](./img/数据集3.png)
 
### 预训练模型

为方便教学，本案例使用**BERT-Base,Uncased**预训练模型。从链接[BERT-Base, Uncased](https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip)下载解压后得到如下预训练模型文件

- bert_config.json
- bert_model.ckpt.data-00000-of-00001
- bert_model.ckpt.index
- bert_model.ckpt.meta
- vocab.txt

其中
* 一个config file(bert_config.json) ：存储预训练模型超参数
* 三个tensorflow checkpoint (bert_model.ckpt) ：包含预训练模型checkpoint
* 一个vocab文件(vocab.txt)：将WordPiece映射成word id  

首先从obs上下载所需数据集、文件及模型的压缩包如下：

In [1]:
from modelarts.session import Session
session = Session()
print("start download squad.tar.gz. Waiting")
session.download_data(bucket_path="modelarts-labs/end2end/NLP/squad.tar.gz", path="./squad.tar.gz")
# 以上步骤在ModelArts Notebook中运行，只有在下载完毕后才有输出

#查看目录
!ls -la  

start download squad.tar.gz. Waiting
Successfully download file modelarts-labs/end2end/NLP/squad.tar.gz from OBS to local ./squad.tar.gz
total 407396
drwsrwsrwx  5 root   users      4096 May 31 04:31 .
drwsrwsr-x 18 jovyan users      4096 May 31 04:31 ..
drwxr-s---  2 jovyan users      4096 May 31 04:31 img
drwxr-s---  2 jovyan users      4096 May 31 04:30 .ipynb_checkpoints
drwsrwsrwx  2 root   users     16384 May 31 04:29 lost+found
-rw-r-----  1 jovyan users    335755 May 31 04:30 SQUAD-BERT.ipynb
-rw-r-----  1 jovyan users 416802816 May 31 04:33 squad.tar.gz


In [2]:
# 使用tar命令解压资源包
!tar xf ./squad.tar.gz

# 使用rm命令删除压缩包
!rm ./squad.tar.gz

#查看目录，与上对比检查是否增加NER文件夹
!ls -la    

total 364
drwsrwsrwx  6 root   users   4096 May 31 04:33 .
drwsrwsr-x 18 jovyan users   4096 May 31 04:31 ..
drwxr-s---  2 jovyan users   4096 May 31 04:31 img
drwxr-s---  2 jovyan users   4096 May 31 04:30 .ipynb_checkpoints
drwsrwsrwx  2 root   users  16384 May 31 04:29 lost+found
drwxr-s---  7 jovyan users   4096 May 31 03:51 squad
-rw-r-----  1 jovyan users 335755 May 31 04:30 SQUAD-BERT.ipynb


解压后文件夹结构如下：

```bash
squad
 ├── bert
 │       ├── modeling.py
 │       ├── optimization.py
 │       ├── run_squad.py
 │       └── tokenization.py
 │
 ├── uncased_L-12_H-768_A-12
 │       ├──bert_config.json
 │       ├──bert_model.ckpt.data-00000-of-00001
 │       ├──bert_model.ckpt.index
 │       ├──bert_model.ckpt.meta
 │       └──vocab.txt
 ├── data
 │       ├── train-v1.1.json  
 │       └── dev-v1.1.json
 ├── src
 │       └── evaluate-v1.1.py 
 └── output
```

本案例所需条件：

tensorflow >= 1.11.0   

tensorflow-gpu  >= 1.11.0 

因此需要首先升级tensorflow和tensorflow-gpu

In [3]:
print("start update tensorflow Waiting......15 sec")
!pip install tensorflow==1.11.0
print("下载完毕后tensorflow需要自动配置，等待15秒再执行后面的内容")

start update tensorflow Waiting......15 sec
    100% |████████████████████████████████| 63.0MB 58.9MB/s ta 0:00:0110:01MB 86.7MB/s eta 0:00:01█████████▏                     | 19.9MB 79.3MB/s eta 0:00:01██▎                    | 22.3MB 40.4MB/s eta 0:00:02████████                 | 29.6MB 99.8MB/s eta 0:00:01eta 0:00:01 54.4MB 72.0MB/s eta 0:00:01ta 0:00:01
    100% |████████████████████████████████| 1.2MB 53.4MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 22.3MB/s ta 0:00:01
    100% |████████████████████████████████| 3.0MB 93.0MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 9.5MB/s ta 0:00:011
  Found existing installation: protobuf 3.5.2
    Uninstalling protobuf-3.5.2:
      Successfully uninstalled protobuf-3.5.2
  Found existing installation: Keras-Applications 1.0.2
    Uninstalling Keras-Applications-1.0.2:
      Successfully uninstalled Keras-Applications-1.0.2
  Found existing installation: tensorboard 1.8.0
    Uninstalling tensorboard-1.8.0

In [4]:
print("start update tensorflow-gpu Waiting.....15 sec")
!pip install tensorflow-gpu==1.11.0
print("下载完毕后tensorflow-gpu需要自动配置，等待15秒再执行后面的内容")

start update tensorflow-gpu Waiting.....15 sec
    100% |████████████████████████████████| 258.8MB 69.2MB/s ta 0:00:011% |█▌                              | 12.3MB 78.8MB/s eta 0:00:0404                         | 18.2MB 74.2MB/s eta 0:00:04                         | 22.2MB 94.6MB/s eta 0:00:03                 | 29.1MB 74.7MB/s eta 0:00:04        | 40.8MB 74.3MB/s eta 0:00:03��█▍                          | 43.6MB 75.3MB/s eta 0:00:03��▊                          | 46.3MB 77.9MB/s eta 0:00:03                     | 52.3MB 84.3MB/s eta 0:00:037.2MB 97.5MB/s eta 0:00:03                | 61.4MB 96.2MB/s eta 0:00:03�▏                       | 66.3MB 92.4MB/s eta 0:00:03 eta 0:00:02                     | 75.7MB 95.4MB/s eta 0:00:02��█████                      | 80.2MB 93.8MB/s eta 0:00:02MB/s eta 0:00:021MB/s eta 0:00:02                   | 94.1MB 98.7MB/s eta 0:00:02          | 103.5MB 85.8MB/s eta 0:00:02███▎                  | 107.4MB 90.1MB/s eta 0:00:02█████▉                  | 111.9MB 102.7

In [5]:
print("打印tensorflow版本，确认正确更新 等待10秒")
import tensorflow as tf
print("tensorflow-version：", tf.__version__)

打印tensorflow版本，确认正确更新 等待10秒
tensorflow-version： 1.11.0


现在，就可以运行run_squad.py文件，注意路径和一些参数的设置。

训练时间较长，请耐心等待

In [6]:
%run squad/bert/run_squad.py \
  --vocab_file=./squad/uncased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=./squad/uncased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=./squad/uncased_L-12_H-768_A-12/bert_model.ckpt \
  --do_train=True \
  --train_file=./squad/data/train-v1.1.json \
  --do_predict=True \
  --predict_file=./squad/data/dev-v1.1.json \
  --train_batch_size=12 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=./squad/output/

INFO:tensorflow:Using config: {'_model_dir': './squad/output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f14e2065320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, inp

INFO:tensorflow:token_is_max_context: 13:True 14:True 15:True 16:True 17:True 18:True 19:True 20:True 21:True 22:True 23:True 24:True 25:True 26:True 27:True 28:True 29:True 30:True 31:True 32:True 33:True 34:True 35:True 36:True 37:True 38:True 39:True 40:True 41:True 42:True 43:True 44:True 45:True 46:True 47:True 48:True 49:True 50:True 51:True 52:True 53:True 54:True 55:True 56:True 57:True 58:True 59:True 60:True 61:True 62:True 63:True 64:True 65:True 66:True 67:True 68:True 69:True 70:True 71:True 72:True 73:True 74:True 75:True 76:True 77:True 78:True 79:True 80:True 81:True 82:True 83:True 84:True 85:True 86:True 87:True 88:True 89:True 90:True 91:True 92:True 93:True 94:True 95:True 96:True 97:True 98:True 99:True 100:True 101:True 102:True 103:True 104:True 105:True 106:True 107:True 108:True 109:True 110:True 111:True 112:True 113:True 114:True 115:True 116:True 117:True 118:True 119:True 120:True 121:True 122:True 123:True 124:True 125:True 126:True 127:True 128:True 129:T

INFO:tensorflow:input_ids: 101 1999 2255 2230 1010 2040 2001 1996 10501 17656 2008 4541 2129 1996 1057 1012 1055 1012 2976 3914 2001 4526 9598 2000 4337 1996 6381 3012 8132 1029 102 2023 4923 13184 2716 1996 3795 3361 2291 2000 1996 20911 1997 7859 1012 1996 3433 1997 1996 2976 3914 1010 1996 2647 2430 2924 1010 1996 2924 1997 2563 1998 2060 2430 5085 2001 6234 1998 6918 1012 2076 1996 2197 4284 1997 2263 1010 2122 2430 5085 4156 2149 1002 1016 1012 1019 23458 1997 2231 7016 1998 11587 2797 7045 2013 5085 1012 2023 2001 1996 2922 6381 3012 13341 2046 1996 4923 3006 1010 1998 1996 2922 12194 3343 2895 1010 1999 2088 2381 1012 2206 1037 2944 7531 2011 1996 2142 2983 2924 5343 7427 1010 1996 6867 1997 2647 3741 1998 1996 3915 12361 1996 7016 3843 2011 2037 5085 1998 2992 1996 3007 1997 2037 2120 8169 3001 1010 4821 13131 1002 1015 1012 1019 23458 4397 3843 6871 4518 1999 2037 2350 5085 1012 1999 2255 2230 1010 10501 17656 3312 2358 8004 24725 4541 2129 1996 1057 1012 1055 1012 2976 3914 2

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

INFO:tensorflow:input_ids: 101 2054 2785 1997 7832 2211 6037 2062 4141 1999 4623 1998 3906 2076 1996 16724 1029 102 1996 3747 1997 2671 2036 2211 6037 2062 4141 1999 4623 1998 3906 2076 1996 16724 1012 2070 4623 2150 29592 2007 4045 19240 1998 13425 1010 2096 2060 5878 2020 2517 3495 2055 4045 7832 1012 2909 2957 2304 5974 5462 1996 8446 2937 2291 2000 7893 1999 4325 1010 1037 9569 5961 1999 2698 2808 1006 28460 1007 1012 2044 8446 1005 1055 2331 1999 25350 1010 5878 2020 3605 1999 2010 6225 2005 5109 1012 2508 11161 1006 16601 1516 24445 1007 17430 2010 1000 5961 2000 1996 3638 1997 8446 1010 1000 2029 9587 21737 2094 1996 3279 1997 8446 1010 2021 2036 5868 2010 2671 1998 8027 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

...

INFO:tensorflow:token_to_orig_map: 12:0 13:1 14:2 15:3 16:4 17:5 18:6 19:7 20:8 21:9 22:10 23:11 24:12 25:13 26:14 27:15 28:16 29:17 30:17 31:17 32:18 33:19 34:20 35:21 36:21 37:22 38:23 39:24 40:25 41:26 42:26 43:26 44:27 45:28 46:29 47:30 48:31 49:32 50:33 51:34 52:35 53:35 54:35 55:36 56:37 57:38 58:39 59:39 60:39 61:40 62:41 63:42 64:43 65:44 66:45 67:46 68:46 69:47 70:48 71:49 72:50 73:51 74:52 75:53 76:53 77:54 78:54 79:55 80:56 81:56 82:56 83:57 84:58 85:59 86:60 87:61 88:62 89:63 90:64 91:65 92:66 93:66 94:67 95:67 96:68 97:69 98:70 99:71 100:72 101:73 102:74 103:74 104:75 105:76 106:77 107:78 108:79 109:79 110:80 111:80 112:81 113:82 114:83 115:83 116:83 117:84 118:84 119:85 120:86 121:87 122:88 123:89 124:89 125:90 126:91 127:92 128:93 129:94 130:95 131:96 132:97 133:98 134:99 135:100 136:100 137:100 138:101 139:101 140:102 141:103 142:104 143:105 144:106 145:107 146:108 147:109 148:110 149:110 150:111 151:112 152:112 153:112 154:112 155:113 156:114 157:115 158:116 159:117 16

INFO:tensorflow:input_ids: 101 2054 3346 2106 3565 4605 2753 2202 2173 1999 1029 102 3565 4605 2753 2001 2019 2137 2374 2208 2000 5646 1996 3410 1997 1996 2120 2374 2223 1006 5088 1007 2005 1996 2325 2161 1012 1996 2137 2374 3034 1006 10511 1007 3410 7573 14169 3249 1996 2120 2374 3034 1006 22309 1007 3410 3792 12915 2484 1516 2184 2000 7796 2037 2353 3565 4605 2516 1012 1996 2208 2001 2209 2006 2337 1021 1010 2355 1010 2012 11902 1005 1055 3346 1999 1996 2624 3799 3016 2181 2012 4203 10254 1010 2662 1012 2004 2023 2001 1996 12951 3565 4605 1010 1996 2223 13155 1996 1000 3585 5315 1000 2007 2536 2751 1011 11773 11107 1010 2004 2092 2004 8184 28324 2075 1996 4535 1997 10324 2169 3565 4605 2208 2007 3142 16371 28990 2015 1006 2104 2029 1996 2208 2052 2031 2042 2124 2004 1000 3565 4605 1048 1000 1007 1010 2061 2008 1996 8154 2071 14500 3444 1996 5640 16371 28990 2015 2753 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:***** Running predictions *****
INFO:tensorflow:  Num orig examples = 10570
INFO:tensorflow:  Num split examples = 10833
INFO:tensorflow:  Batch size = 8
INFO:tensorflow:Calling model_fn.

INFO:tensorflow:  name = bert/encoder/layer_3/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder

INFO:tensorflow:  name = bert/encoder/layer_8/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflo

运行成功后，在output文件夹下会输出一个predictions.json文件，执行src目录下的
[evaluate-v1.1.py](https://github.com/allenai/bi-att-flow/blob/master/squad/evaluate-v1.1.py)文件，可以得到验证集的预测指标，如下：

In [7]:
%run squad/src/evaluate-v1.1.py squad/data/dev-v1.1.json squad/output/predictions.json

{"exact_match": 80.82308420056765, "f1": 88.35275036741817}


输出验证集的第一个问题所在**文章标题、段落、问题、标准答案**，并输出预测的答案，如下：

In [8]:
import json
with open("./squad/data/dev-v1.1.json") as dev_file:
    devs = json.load(dev_file)
    title = devs['data'][0]['title']
    context = devs['data'][0]['paragraphs'][0]['context']
    q = devs['data'][0]['paragraphs'][0]['qas'][0]['question']
    a = devs['data'][0]['paragraphs'][0]['qas'][0]['answers']
    print('题目:',title)
    print('段落:',context)
    print('问题:',q)
    print('答案:',a)

with open("./squad/output/predictions.json") as prediction_file:
    predictions = json.load(prediction_file)
    for key in list(sorted(predictions.keys())):
        print('\n预测答案:',predictions[key])
        break

题目: Super_Bowl_50
段落: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
问题: Which NFL team represented the AFC at Super Bowl 50?
答案: [{'answer_start': 177, 'text': 'Denver Broncos'}, {'answer_start': 177, 'text': 'Denver Broncos'}, {'answer_start': 177, 'text': 'Denver Bron

得到如上结果，即使用BERT模型的首个程序运行成功。

### <span style="color:red">注意，现阶段ModelArts创建环境5G以下的EVS配置免费，超过请在实践项目结束后停止并删除该开发环境，以免产生多余费用。</span>

